In [1]:
import gurobipy as grb
import numpy as np
import itertools
import sys
import copy
import collections
import functools

In [2]:
def iterer(*args):
    return itertools.product(*[x_ if isinstance(x_,collections.Iterable) else range(x_) for x_ in args])

In [3]:
possible_plans = [
        ['orange','black','green'],
        ['blue','black','yellow'],
        ['blue','green','orange'],
        ['yellow','green','black'],
        ['black','yellow','orange'],
        ['green','yellow','blue'],
        ['blue','orange','black'],
        ['green','orange','yellow'],
        ['black','blue','green'],
        ['orange','blue','yellow']
    ]

colormap = {
    'black'  : 0,
    'green'  : 1,
    'blue'   : 2,
    'orange' : 3,
    'yellow' : 4
}

In [5]:
"""
    places in heaps: (1 is closer to orange side, 0 to central tracking device)
       | 1|
    | 0| 4| 2|
       | 3|

    colors of cubes: (same as for orange side)
    
    black  -- 0
    green  -- 1
    blue   -- 2
    orange -- 3
    yellow -- 4
    
    numbers of heaps - 0,1,2,3,4,5 CCW starting from closest heap to central tracking device on the orange side
    
    man = 0, 1, 2 (picking 1 4 3 if from 0 for orange!)
    
    cubes:
    -1 - none
    
""";

In [6]:
class STNode():
    def __init__(self, state, upper):
        self.state = state
        self.upper = upper
        self.nexts = []
        
    def __str__(self):
        return str(self.state)
    
    def gen_reversed_path(self):
        sn = self
        while sn != None:
            yield sn
            sn = sn.upper
    
    def get_path(self):
        return reversed(list(self.gen_reversed_path()))

In [365]:
class StateFull():
    X = [0,1,2]
    profile_reduce = {
        0:0,
        1:1,
        2:0,
        3:0,
        4:4,
        -1:0
    }
    initial_cubes = np.array([[-1,-1,-1,-1,-1],
                                [-1,-1,-1,-1,-1],
                                [-1,-1,-1,-1,-1]])
    
    def __init__(self, lines, initial_profile = np.array([-1,-1,-1]), x = 0, y = 0, time = 0):
        self.lines = lines
        self.cubes = self.initial_cubes
        self.cubes[:,0] = initial_profile
        self.heights = 1*(initial_profile > -1)
        self.x = x
        self.y = y
        self.time = time
        self.n_picks = 0
        
    def get_next_line(self):
        for x, l in zip(self.X, self.lines):
            any_cube = False
            for c in l:
                if c != -1:
                    any_cube = True
            if any_cube:
                return x,l
    
    def pick_cube_by(self, color, man):
        if self.heights[man] < 5:
            self.cubes[man,self.heights[man]] = color
            self.heights[man] += 1
    
    def __str__(self):
        s = "---STATE---" + '\n'
        for l in self.lines:
            s += str(l) + '\n'
        s += "x=%d, y=%d, time=%f, %d picks total\n"%(self.x, self.y, self.time, self.n_picks)
        s += "height = " + str(self.cubes) + '\n'
        s += '-----------'
        return s
    
    # def get_key(self):
    #     return str(self.heights + [self.profile_reduce[p] for p in self.profile])
    
    def is_final(self):
        # if all is -1
        return np.all(self.lines < 0)

In [455]:
class SuctionRobotStrategyOptimizerFull():
    defaults = {
        'plan': [0,1,2],
        'allowed_sides': [
            [False, True, True, True],
            [True, False, True, True],
            [True, True, False, True],
            [True, True, False, True],
            [True, True, False, True],
            [True, True, True, False]
        ],
        'heaps': [0,1,2],
        'pick_time': 2,
        'move_time': 1,
        'heap': {'orange':np.array([
                        [-1, 0,-1],
                        [ 1, 4, 3],
                        [-1, 2,-1]
                    ]),
                 'green':np.array([
                        [-1, 0,-1],
                        [ 3, 4, 1],
                        [-1, 2,-1]
                 ])
            }
    }
    def __init__(self, **kvargs):
        self.params = {}
        for k,v in self.defaults.items():
            self.params[k] = copy.copy(v)
            
        for k,v in kvargs.items():
            if k in self.params.keys():
                self.params[k] = copy.copy(v)
                
        self.manipulator_masks = [list(x) for x in itertools.product([0,1], repeat=3)][1:]
        
    def rotate_heap(self, heap, side):
        return np.rot90(heap, side)
    
    def get_heap(self, number = 0, side = 0):
        heap = None
        if number in [0,1,2]:
            heap = self.params['heap']['orange']
        if number in [3,4,5]:
            heap = self.params['heap']['green']
        return self.rotate_heap(heap,side)
    
    def set_plan(self, plan):
        if isinstance(plan[0], str):
            self.params['plan'] = [colormap[c] for c in plan]
        else:
            self.params['plan'] = plan
        
    def get_plan(self, num = None):
        if num == None:
            return self.params['plan']
        elif num in [0,1,2]:
            return self.params['plan'][num]
        return self.params['plan']
    
    def reverse_plan(self):
        self.params['plan'] = self.params['plan'][::-1]
        
    def get_move_time(self, dx, dy):
        return max(abs(dx),abs(dy))*self.params['move_time']
    
    def get_lines_by_plan_and_heap(self, heap):
        lines = copy.copy(heap)
        for i, j in iterer(3,3):
            if heap[i][j] in range(5):
                if heap[i][j] in self.get_plan():
                    lines[i][j] = self.get_plan().index(heap[i][j]) + 1
                else:
                    lines[i][j] = 0
        return lines
    
    def try_to_pick(self, state, y, l, x, mans):
        picked = [-1,-1,-1]
        for i in range(3): # run over mans
            if mans[i] == 1: # if need to pick by i-th man
                if i + y in [0,1,2] and l[i+y] >= 0: # check if we pick existing cube
                    # print(i,y)
                    state.pick_cube_by(l[i+y],i)
                    state.lines[x][i+y] = -1
                else:
                    return None
        # if still not return, add time and return new state
        state.time += self.params['pick_time']
        state.time += self.get_move_time(x-state.x, y-state.y)
        state.n_picks += 1
        state.x = x
        state.y = y
        return state
    
    def get_new_states(self, state):
        if not state.is_final():
            x, l = state.get_next_line()
            for mans, y in iterer(self.manipulator_masks, range(-2,3)):
                new_state = self.try_to_pick(copy.deepcopy(state), y, l, x, mans)
                if new_state != None:
                    # print(y, mans)
                    yield new_state
                    
    def reduce_final_states(self, final_states):
        # we can choose only same heights with best times
        heights = {}
        for sn in final_states:
            s = sn.state
            key = str(s.cubes)
            if key in heights:
                if heights[key].state.time > s.time:
                    heights[key] = sn
            else:
                heights[key] = sn
        return list(heights.values())
    
    def get_all_cube_profiles(self, final_states):
        profiles = {}
        for sn in final_states:
            s = sn.state
            key = str(s.cubes)
            if key in profiles:
                if profiles[key].state.time > s.time:
                    profiles[key] = sn
            else:
                profiles[key] = sn
        return list(profiles.keys())
    
    def checker_node_for_plans(self):
        def f(sn):
            return self.check_state_for_plans(sn.state)
        return f
    
    def reduce_by_colors(self, final_states):
        extra_colors = set([0,1,2,3,4]) - set(opt.get_plan())
        cubes = {}
        for sn in final_states:
            s = sn.state
            for ec in extra_colors:
                s.cubes[s.cubes == ec] = 6
            key = str(s.cubes)
            if key in cubes:
                if cubes[key].state.time > s.time:
                    cubes[key] = sn
            else:
                cubes[key] = sn
        return list(cubes.values())
    
    def build_solution_tree(self, heap_num, side, only_good=False):
        h = self.get_heap(heap_num, side)
        initial_state = StateFull(h)
        root = STNode(initial_state, None)
        this_wave = [root]
        next_wave = []
        N = 0
        final_states = []
        while len(this_wave) > 0:
            for node in this_wave:
                if node.state.is_final():
                    final_states.append(node)
                node.nexts = [STNode(x,node) for x in self.get_new_states(node.state)]
                next_wave.extend(node.nexts)
            if len(next_wave) > 0:
                N = len(next_wave)
            this_wave = next_wave
            next_wave = []
        
        fss = self.reduce_final_states(final_states)
        fss2 = self.reduce_by_colors(fss)
        
        if only_good:
            return root, list(filter(self.checker_node_for_plans(), fss2))
        
        return root, fss2
    
   
    
    def count_all_variants(self, **kvargs):
        team_colors, sides = 2, 4
        if 'heap_num' in kvargs:
            team_colors = [kvargs['heap_num'] // 3]
        if 'use_sides_constraint' in kvargs and kvargs['use_sides_constraint'] and 'heap_num' in kvargs:
            sides = [side for allowed, side in zip(self.params['allowed_sides'][kvargs['heap_num']],list(range(4))) if allowed ]

        variants = []
        progress = 0
        for team_color, side in iterer(team_colors, sides):
            _, final_states  = self.build_solution_tree(team_color*3, side, True)
            for fs in final_states:
                variants.append([team_color, side, fs])
            progress += 1 
        return variants
   
    def is_equal_to_plan(self, _tp):
        plan = self.get_plan()
        tp = copy.deepcopy(_tp)
        tp2 = np.array(list(reversed(tp)))
        for i, c in enumerate(tp):
            if c == 5:
                tp[i] = plan[i]
        for i, c in enumerate(tp2):
            if c == 5:
                tp2[i] = plan[i]
        return np.all(tp==plan) or np.all(tp2==plan)
    
    def check_state_for_plans(self, st):
        np_plan =np.array(opt.get_plan())
        np_rev_plan = np.array(list(reversed(opt.get_plan())))
        has_plan = np.array([False, False, False])
        for j, tower in enumerate(st.cubes):
            for i in range(3):
                if self.is_equal_to_plan(tower[i:i+3]):
                    has_plan[j] = True
        return sum(1*has_plan)
    
    def check_for_3plans(self, variants):
        st = copy.deepcopy(variants[0][-1].state)
        for man in range(3):
            for v in variants[1:]:
                for c in v[-1].state.cubes[man]:
                    if c >= 0:
                        st.pick_cube_by(c, man)
        return self.check_state_for_plans(st) == 3
    
    def check_for_3plans_with_starting_profile(self, variants, profile):
        st = StateFull(opt.get_heap(),profile)
        for man in range(3):
            for v in variants:
                for c in v[-1].state.cubes[man]:
                    if c >= 0:
                        st.pick_cube_by(c, man)
        return self.check_state_for_plans(st) == 3
    
    def all_heights_up_to_5(self, heights):
        total = sum(heights)
        rest = 5 - total
        if rest == 0:
            yield heights
        if rest == 1:
            for dh in [[1,0,0],[0,1,0],[0,0,1]]:
                print(dh)
                yield heights + np.array(dh)
        if rest == 2:
            for dh in [[1,0,0],[0,1,0],[0,0,1]]:
                yield heights + np.array(dh)
            for dh in [[1,1,0],[0,1,1],[1,0,1]]:
                yield heights + np.array(dh)
        
    def count_all_picking_plans(self,heap_nums = [0,1,2], **kvargs):
        variants = [self.count_all_variants(heap_num=hn, use_sides_constraint=True) for hn in heap_nums]
        
        heights = {}
        for v in variants[-1]:
            key = str(v[-1].state.heights)
            if key in heights:
                heights[key].append(v)
            else:
                heights[key]= [v]
                    
        path_variants = []
        i = 0
        total = len(variants[0])*len(variants[1])
        print(total)
        goal_height = np.array([5,5,5])
        for v1,v2 in iterer(variants[0],variants[1]):
            height_left = goal_height - v1[-1].state.heights - v2[-1].state.heights
            for h in self.all_heights_up_to_5(height_left):
                key = str(h)
                if key in heights:
                    #print(len(heights[key]))
                    for v3 in heights[key]:
                       # if self.check_for_3plans([v1,v2,v3]):
                        path_variants.append([v1,v2,v3])
            i+=1
            if i % 1000 == 0:
                print("%d%s" % (i / total *100, '%')) 
        return path_variants
    
    def check_heights(self,vs):
        heights = np.array([0,0,0])
        for v in vs:
            heights += v[-1].state.heights
        return np.all(heights >= 5)
    
    def count_all_picking_plans_simple(self, heap_nums=[0,1,2], **kvargs):
        profile = np.array([-1,-1,-1])
        if 'profile' in kvargs:
            profile = kvargs['profile']
        variants = [self.count_all_variants(heap_num=hn, use_sides_constraint=True) for hn in heap_nums]
                    
        path_variants = []
        i = 0
        total = len(variants[0])**3
        proc = 0
        print(total)
        for vs in iterer(*variants):   
            if self.check_heights(vs):
                #if self.check_for_3plans_with_starting_profile(vs,profile) != self.check_for_3plans(vs):
                #    print(1111)
                #if self.check_for_3plans_with_starting_profile(vs,profile):
                #    path_variants.append(vs)
                if self.check_for_3plans(vs):
                    path_variants.append(vs)
            i+=1
            if i*100//total != proc:
                proc = i*100//total
                print("%d%s"%(proc,"%"))
        return path_variants
    
    def count_best_picking_plan(self, heap_nums = [0,1,2]):
        path_variants = self.count_all_picking_plans(heap_nums)
        min_v = path_variants[0]
        print(min_v)
        min_t = sum([v[-1].state.time for v in min_v])
        for pv in path_variants:
            pv_t = sum([v[-1].state.time for v in pv])
            if pv_t < min_t:
                min_v = pv
                min_t = pv_t
        return min_v
    
    def get_heap_path(self, variant):
        root, final_states = self.build_solution_tree(v[0]*3,v[1],v[2])
        route = []
        for fs in final_states:
            if fs.state.heights == v[-1].state.heights:
                sn = fs
                while sn != None:
                    route.append(sn)
                    sn = sn.upper
                break
        return reversed(route)
    

In [456]:
opt = SuctionRobotStrategyOptimizerFull()
opt.set_plan(possible_plans[2])
print(possible_plans[2])
opt.get_plan()

['blue', 'green', 'orange']


[2, 1, 3]

In [453]:
testing_triple = [0,1,0]
print((opt.get_heap(*testing_triple[:2])))

[[-1  3 -1]
 [ 0  4  2]
 [-1  1 -1]]


In [432]:
root, final_states = opt.build_solution_tree(*[0,2])

In [433]:
len(final_states)

396

In [425]:
vs = opt.count_all_variants()

In [183]:
vs[0][-1].state.heights + vs[4][-1].state.heights + vs[26][-1].state.heights

array([5, 5, 5])

In [182]:
for i in range(0,100):
    if np.all(vs[0][-1].state.heights + vs[4][-1].state.heights + vs[i][-1].state.heights - 5 == 0):
        print(i)
        break

26


In [184]:
opt.check_for_3plans([vs[0],vs[4],vs[26]])

True

In [427]:
%%time
pv = opt.count_all_picking_plans_simple()

154854153
1%
2%
3%
4%
5%
6%
7%
8%
9%
10%
11%
12%
13%
14%
15%
16%
17%
18%
19%
20%
21%
22%
23%
24%
25%
26%
27%
28%
29%
30%
31%
32%
33%
34%
35%
36%
37%
38%
39%
40%
41%
42%
43%
44%
45%
46%
47%
48%
49%
50%
51%
52%
53%
54%
55%


KeyboardInterrupt: 

In [323]:
len(pv)

1080

In [457]:
results = []
for p in possible_plans:
    opt.set_plan(p)
    best_pv = opt.count_best_picking_plan([2,1,0])
    #for v in best_pv:
    #    print(v[:3],v[-1])
    results.append(best_pv)
    total_time = sum([v[-1].state.time for v in best_pv])
    total_picks = sum([v[-1].state.n_picks for v in best_pv])
    print(total_time, total_picks)

321895
0%
0%
0%
1%
1%
1%
2%
2%
2%
3%
3%
3%
4%
4%
4%
4%
5%
5%
5%
6%
6%
6%
7%
7%
7%
8%
8%
8%
9%
9%
9%
9%
10%
10%
10%
11%
11%
11%
12%
12%
12%
13%
13%
13%
13%
14%
14%
14%
15%
15%
15%
16%
16%
16%
17%
17%
17%
18%
18%
18%
18%
19%
19%
19%
20%
20%
20%
21%
21%
21%
22%
22%
22%
22%
23%
23%
23%
24%
24%
24%
25%
25%
25%
26%
26%
26%
27%
27%
27%
27%
28%
28%
28%
29%
29%
29%
30%
30%
30%
31%
31%
31%
31%
32%
32%
32%
33%
33%
33%
34%
34%
34%
35%
35%
35%
36%
36%
36%
36%
37%
37%
37%
38%
38%
38%
39%
39%
39%
40%
40%
40%
41%
41%
41%
41%
42%
42%
42%
43%
43%
43%
44%
44%
44%
45%
45%
45%
45%
46%
46%
46%
47%
47%
47%
48%
48%
48%
49%
49%
49%
50%
50%
50%
50%
51%
51%
51%
52%
52%
52%
53%
53%
53%
54%
54%
54%
54%
55%
55%
55%
56%
56%
56%
57%
57%
57%
58%
58%
58%
59%
59%
59%
59%
60%
60%
60%
61%
61%
61%
62%
62%
62%
63%
63%
63%
63%
64%
64%
64%
65%
65%
65%
66%
66%
66%
67%
67%
67%
68%
68%
68%
68%
69%
69%
69%
70%
70%
70%
71%
71%
71%
72%
72%
72%
73%
73%
73%
73%
74%
74%
74%
75%
75%
75%
76%
76%
76%
77%
77%
77%
77%
78%
78%
78%
79%
79%
7

['orange', 'black', 'green']

In [459]:
for v in results[0]:
    print(v[-1].state)

---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=1, time=8.000000, 3 picks total
height = [[1 6 0 6 3]
 [0 6 0 3 1]
 [3 1 0 3 6]]
-----------
---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=1, time=8.000000, 3 picks total
height = [[1 6 0 6 3]
 [0 6 0 3 1]
 [3 1 0 3 6]]
-----------
---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=-1, time=9.000000, 3 picks total
height = [[3 6 0 6 3]
 [6 6 0 3 1]
 [6 1 0 3 6]]
-----------


In [445]:
best_pv[-1]

[0, 1, <__main__.STNode at 0x7f5aae14d128>]

In [331]:
for v in bv:
    print(v[-1].state)

---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=-1, time=13.000000, 4 picks total
height = [[-1 -1 -1 -1 -1]
 [ 4 -1 -1 -1 -1]
 [ 2  1  3  0 -1]]
-----------
---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=-1, time=12.000000, 4 picks total
height = [[ 4 -1 -1 -1 -1]
 [ 2  1  3 -1 -1]
 [ 0 -1 -1 -1 -1]]
-----------
---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=1, time=13.000000, 4 picks total
height = [[ 0  3  1  2 -1]
 [ 4 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1]]
-----------


In [310]:
for v in iterer(2,2,2):
    print(v)

(0, 0, 0)
(0, 0, 1)
(0, 1, 0)
(0, 1, 1)
(1, 0, 0)
(1, 0, 1)
(1, 1, 0)
(1, 1, 1)


In [409]:
a = np.array([0,1,2,1])

In [413]:
a

array([0, 6, 2, 6])